In [ ]:
try:
    # se estiver no colab vai rodar o bloco abaixo
    from google.colab import data_table, drive
    drive.mount('/content/drive', force_remount=True)

    data_table.enable_dataframe_formatter()
    data_table.DataTable.max_columns = 50
    path = "/content/drive/MyDrive/doutorado"  # caminho do google drive
except:
    # se nao estiver no colab vai rodar o bloco abaixo
    path = "."  # caminho local

In [ ]:
# carrega bibliotecas
import pandas as pd
import numpy as np
pd.options.mode.chained_assignment = None

In [ ]:
# carrega dados do .csv
dados = pd.read_csv(path + "/dados_brasil_py.csv", sep=";", index_col=0, parse_dates=True, na_values=["//"], skipinitialspace=True, encoding="unicode_escape")

dados = dados.reset_index()  # reset index transforma "timestamp" em coluna comum, para fazer os cálculos de tempo
dados.columns = dados.columns.str.strip()  # remove espaços em branco dos títulos das colunas
dados = dados.apply(lambda x: x.str.strip() if x.dtype == "object" else x)  # remove espaços em branco do conteudo dos dados

ix = dados.reset_index().index[dados["stroke_order"] == "a"].tolist()  # procura o indice da linha em que "stroke_order" == "a"
dados = [dados.iloc[a:b] for a, b in zip(ix, ix[1:] + [None])]  # cria uma lista com o subset das linhas entre o primeiro "a" e o proximo "a" (ultimo "a" não incluso)

for r_idx, raio in enumerate(dados):
    dados[r_idx]["raio"] = r_idx  # cria uma coluna "raio" com o numero do raio atual
    dados[r_idx]["nstroke"] = range(1, len(raio) + 1)  # cria uma coluna "nstroke" com o numero do stroke
    dados[r_idx]["n_strokes"] = len(raio)  # cria (atualiza) a coluna "n_strokes" com o numero total de strokes

dados = pd.concat(dados)  # recria o dataframe com todos os raios e as novas colunas

print("Total de descargas:", len(dados))

dados.head()  # mostra as primeiras linhas do dataframe

In [ ]:
descargas = dados.copy()  # copia os "dados" em "descargas"

descargas['subseq'] = ""  # Acrescenta a coluna 'subseq'
descargas['after'] = ""  # Acrescenta a coluna 'after'
descargas['where'] = ""  # Acrescenta a coluna 'where'
descargas['raio_duracao'] = ""  # Acrescenta a coluna 'raio_duracao'

# Aplica filtros de local, distancia, polaridade, etc
# descargas = descargas.groupby("raio").filter(lambda r: r["local"].isin(["Utah"]).any()) # selecionando locais
descargas = descargas.groupby("raio").filter(lambda r: ~r["local"].isin(["Utah"]).any())  # excluindo Utah
# descargas = descargas.groupby("raio").filter(lambda r: r["year"].between(2003, 2011).any()) #seleciona o período
# descargas = descargas.groupby("raio").filter(lambda r: r["dist1"].between(0, 30, inclusive="right").any()) #seleciona distancia
descargas = descargas.groupby("raio").filter(lambda r: not r["visib"].isin(["m", "f"]).any())  # exclui as visibilidades
descargas = descargas.groupby("raio").filter(lambda r: not r["contact"].isin(['0']).any())  # exclui ponto de contato igual a zero
descargas = descargas.groupby("raio").filter(lambda r: r["Ip1"].lt(0).any() and not r["Ip1"].gt(0).any())  # negativos
# descargas = descargas.groupby("raio").filter(lambda r: r["Ip1"].gt(0).any() and not r["Ip1"].lt(0).any()) # positivos
# descargas = descargas.groupby("raio").filter(lambda r: r["Ip1"].lt(0).any() and r["Ip1"].gt(0).any()) # bipolares

descargas["CC_duration"] = descargas["CC_duration"].replace(0.0, np.nan)

# intervalo entre descargas
descargas["stroke_intv"] = descargas["timestamp"] - descargas["timestamp"].shift()  # timestamp atual menos o timestamp anterior
descargas["stroke_intv"] = descargas["stroke_intv"].dt.total_seconds() * 1000  # converte para ms
descargas.loc[descargas["nstroke"] == 1, "stroke_intv"] = np.nan  # onde nstroke == 1, invalida o valor de stroke_intv

# ausencia de cc entre descargas
descargas["no_cc"] = (descargas["sec.ms"] - (descargas["sec.ms"] + descargas['CC_duration']/1000).shift())*1000
descargas["no_cc"] = descargas["no_cc"].round(0).fillna(0.0).astype(int)
descargas.loc[descargas["nstroke"] == 1, "no_cc"] = np.nan  # onde nstroke == 1, invalida o valor de stroke_intv

# variáveis para cálculo de parâmetros do stroke_intv
stroke_intv_a = (descargas[descargas["CC_duration"].between(0, 10)]["stroke_intv"])
stroke_intv_b = (descargas[descargas["CC_duration"].between(10, 40)]["stroke_intv"])
stroke_intv_c = (descargas[descargas["CC_duration"] > 40]["stroke_intv"])

# limitando os valores do stroke_intv
# stroke_intv_a = (descargas[descargas["CC_duration"].between(0 , 10)]["stroke_intv"].between(0, 100)).mean()
# stroke_intv_b = (descargas[descargas["CC_duration"].between(10 , 40)]["stroke_intv"].between(0, 100)).mean()
# stroke_intv_c = (descargas[descargas["CC_duration"] > 40]["stroke_intv"].between(0, 100)).mean()

# #Salva o arquivo 'descargas'
# descargas.to_csv("descargas_nocc.csv", index=False)

In [ ]:
raios = descargas.groupby("raio")  # agrupa por raio para calcular as estatisticas e outros parâmetros

raios_min = raios.min(numeric_only=True)  # valores minimos
raios_max = raios.max(numeric_only=True)  # valores maximos
raios_sum = raios.sum(numeric_only=True)  # soma dos valores
raios_first = raios.nth(0)  # primeiro stroke
raios_last = raios.nth(-1)  # ultimo stroke
raios_count = raios.count()  # total elementos no grupo
raios_mean = raios.mean(numeric_only=True)
raios_med = raios.median(numeric_only=True)

raios_sum["n_strokes"] = raios_last["n_strokes"]  # reset no n_strokes da soma

raios_cc = raios.filter(lambda r: r["CC_duration"].ge(3).any())
raios_cc = raios_cc.groupby(["raio"])

raios_ccl = raios.filter(lambda r: r["CC_duration"].gt(40).any())
raios_ccl = raios_ccl.groupby(["raio"])

raios_ccl_last = raios.filter(lambda r: r["CC_duration"].iloc[-1] > 40)
raios_ccl_last = raios_ccl_last.groupby(["raio"])

# abaixo, raio_duracao = inicio_ultimo_stroke + duracao_ultimo_stroke - inicio_primeiro_stroke
# raio_duracao = raios.nth(-1)["timestamp"] + pd.to_timedelta(raios.nth(-1)["CC_duration"], unit='ms') - raios.nth(0)["timestamp"]

descargas_cc = descargas[descargas["CC_duration"].gt(2)]
descargas_cc_muito_curta = descargas[descargas["CC_duration"].between(3, 10, inclusive="both")]
descargas_cc_curta = descargas[descargas["CC_duration"].between(10, 40, inclusive="right")]
descargas_cc_longa = descargas[descargas["CC_duration"].gt(40)]

print("Raios:", raios.ngroups)
print("Raios CCL:", raios_ccl.ngroups)
print("Raios CC:", raios_cc.ngroups)
print("Raios CCL Last:", raios_ccl_last.ngroups)
print("Descargas:", raios.size().sum())
print("Descargas CC:", len(descargas_cc))
print("Descargas CC muito curta:", len(descargas_cc_muito_curta))
print("Descargas CC curta:", len(descargas_cc_curta))
print("Descargas CC longa:", len(descargas_cc_longa))

In [ ]:
contact = [raio for _, raio in raios]  # converte o groupby de df em uma lista de df

# loop em cada raio (grupo de strokes)
for r_idx, raio in enumerate(contact):
    # abaixo, raio_duracao = inicio_ultimo_stroke + duracao_ultimo_stroke - inicio_primeiro_stroke
    raio_duracao = raio.iloc[-1]["timestamp"] + pd.to_timedelta(raio.iloc[-1]["CC_duration"], unit='ms') - raio.iloc[0]["timestamp"]
    d_idx = contact[r_idx].index[0]
    descargas.at[d_idx, 'raio_duracao'] = round(raio_duracao.total_seconds() * 1000)

    if len(raio) < 2:  # Para 'n_strokes' maior ou igual a 2. Se 'nstroke' = 1, não preencha nada.
        continue

    raio.reset_index(inplace=True, drop=True)

    # Preencha 'subseq' da seguinte forma:
    for s_idx, stroke in raio.iterrows():  # loop em cada stroke do raio para preencher o subseq
        if s_idx == 0:  # se for o primeiro stroke, não faz nada
            continue

        # Observe, dentro do mesmo raio, a linha abaixo
        s_curr = contact[r_idx].iloc[s_idx]["contact"]  # stroke atual
        s_prev = contact[r_idx].iloc[s_idx - 1]["contact"]  # stroke anterior
        if s_curr == s_prev:
            # se o número for igual ao anterior, então imprima na coluna 'subseq' a palavra 'same'.
            contact[r_idx].at[s_idx, "subseq"] = "same"
        else:
            # se o número for diferente do anterior, então imprima em 'subseq' a palavra 'new'.
            contact[r_idx].at[s_idx, "subseq"] = "new"

    # Preencha 'after' da seguinte forma:
    for s_idx, stroke in raio.iterrows():  # loop em cada stroke do raio para preencher o after
        if stroke["CC_duration"] <= 40:  # Apenas para 'CC_duration' maior que 40.
            continue

        if stroke["nstroke"] == stroke["n_strokes"]:
            # Se 'nstroke' = 'n_strokes' imprima em 'after' o valor '0'.
            contact[r_idx].at[s_idx, "after"] = 0
        else:
            # Se 'nstroke' for diferente de 'n_strokes', então:
            # Observe a coluna 'subseq', se houver algo escrito, imprima '1' em 'after'
            # obs: sempre terá algo em subseq, então, after = 1
            contact[r_idx].at[s_idx, "after"] = 1

    # Preencha 'where' da seguinte forma. Haverá no final not, new ou same.
    for s_idx, stroke in raio.iterrows():  # loop em cada stroke do raio para preencher o where
        if stroke["after"] == 0:
            # Se 'after' = '0', então imprima 'not'
            contact[r_idx].at[s_idx, "where"] = "not"
        elif stroke["after"] == 1:
            # Se 'after' = '1', então observe a linha imediatamente seguinte em 'subseq',
            if contact[r_idx].iloc[s_idx + 1]["subseq"] == "same":
               # se for 'same', imprima em 'where' a palavra 'same'
                contact[r_idx].at[s_idx, "where"] = "same"
            else:
                # se a linha seguinte for 'new', imprima em 'after' a palavra 'new'.
                contact[r_idx].at[s_idx, "where"] = "new"

contact = pd.concat(contact)  # converte de volta a lista de df para um df completo

contact.to_csv("nocc_contact.csv", index=False)  # Salva o arquivo 'contact'

In [ ]:
nocc_contact = descargas  # copia as descargas
nocc_contact = nocc_contact.groupby("raio").filter(lambda r: r["n_strokes"].gt(1).all())  # filtra
nocc_contact = nocc_contact.groupby("raio").filter(lambda r: r["contact"].eq("1").all())  # filtra
nocc_contact
nocc_contact.to_csv("nocc_contact.csv", index=False)  # Salva o arquivo

In [ ]:
# acrescenta no conjunto de dados as colunas cc_total e cc_max com os valores correspondentes
descargas['cc_total'] = raios_sum["CC_duration"]
descargas['cc_max'] = raios_max["CC_duration"]
descargas.to_csv("nocc_contact.csv", index=False)